# Version 3 

This code uses the variable_kde class (in the dens_estimation module), which is an adapted version from the Scipy 
kernel density estimation code (scipy/scipy/stats/kde.py).

In [1]:
import pandas as pd
import json
import numpy as np
from scipy.stats import multivariate_normal
from scipy import signal
import copy
from importlib import reload
import dens_estimation as de
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

## Import and rebuild data set

Below we import the fitted positions data and build a Pandas DataFrame called 'frame'. In 'frame' each row is a fitted position. The columns contain the separate pieces of information accompanying each fit, such as the coordinates, timestamp, and uncertainty values. The rows are ordered by timestamp.

# real dataset
#F:/ArenaData/arena_fits/2015-07-05.json
data = []
with open("F:/Arena_sim_data/positions_size_20000.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

frame = pd.DataFrame.from_dict(json_lines)

# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

#chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
#chi2PerDof.columns = ['chi2PerDof']
#probChi2 = pd.DataFrame(list(trackee['probChi2']))
#probChi2.columns = ['probChi2']
#nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
#nMeasurements.columns = ['nMeasurements']
localMac = pd.DataFrame(list(trackee['localMac']))
localMac.columns = ['localMac']

# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
#frame = frame.join(chi2PerDof)
#frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(localMac)
#frame = frame.join(nMeasurements)
#del frame['regionsNodesIds']
del frame['error']
#del frame['type']

frame = frame[frame['localMac'] == 0]

frame = frame.sort_values(by='measurementTimestamp')

## Test data set

In [2]:
import uuid

In [3]:
# create artificial wi-fi dataframe for e.g. 4 mobile devices
# play around with the positions and errors, see what happens

data = {'sourceMac': [str(uuid.uuid4()) for i in range(4)],
       'measurementTimestamp': [i + 1436047367297 for i in range(1,5)],
       'coordinates': [[0,0],[-50,-33],[45,-28],[50,28]],
       'x_error': [5,5,5,5],
       'y_error': [5,5,5,5]}

#frame = pd.DataFrame(data)

In [4]:
#frame

# Start data analysis code

The density estimation code works according to the following steps:
- First a time window is selected from the data
- The set of unique MAC addresses in the time window is determined
- A bunch of dictionairies are created which hold for each MAC (as key), the required values to construct
the density estimate, such as the fitted position coordinates, and the associated uncertainty values
- After the first time window, the dictionairies are recreated from the previous set, so the set of MAC addresses
can only expand
- The density estimates are calculated (using the variable_kde class code)
- The density estimates are summed, to create the total crowd density estimate
- After each iteration, a deep copy is made of all the density estimates 
- If a MAC address is not detected in a new time window, the previous density estimate is used (stored in the deep copy)
    - This is only done if the history value associated with the MAC address does not exceed the memory parameter
    - If the history value does exceed the memory parameter, the density estimate remains zero until the MAC is detected again

From here it is assumed that the data is stored in a Pandas DataFrame called 'frame'.
The function 'selectWindow' selects the part of the dataset with timestamps falling within the interval specified by variables 'start' and 'stop', and returns a DataFrame with the same structure as 'frame'.
Start and stop are specified by the iterator k and the parameters 'interval' and 'timestep'.
If timestep > interval, the time windows are non-overlapping.

In [5]:
def selectWindow(frame, k, init_timestamp, timestep, interval):
    #start = min(frame['measurementTimestamp']) + k * timestep
    start = init_timestamp + k * timestep
    stop = start + interval

    window = frame[(frame['measurementTimestamp'] >= start) & 
                       (frame['measurementTimestamp'] < stop)]

    return window

The function createDataStructures returns a bunch of dictionairies for the MAC addresses detected in the 
selected time window, required to do the density estimation later on.
It creates a Python dictionairy called 'histos' with the MAC addresses as keys. Each address gets an empty grid (zeros), 
which is the two-dimensional probability distribution yet to be evaluated.
It creates a second dictionairy called 'positions' where each MAC address gets an empty list.
It creates two separate dictionairies for the uncertainty values in x and y direction,
where each MAC address gets an empty list.
It creates a dictionairy called 'history', which for each MAC keeps track of the time that has passed since the last update, given by the number of time windows.

In [6]:
def createDataStructures(window, height, width):
    grids = np.zeros((len(set(window['sourceMac'])), height,width))

    # dictionary of histograms (with mac addresses as keys)
    histos = dict(zip(set(window['sourceMac']), grids))
    
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    positions = dict(zip(set(window['sourceMac']), emptylist))
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    x_errors = dict(zip(set(window['sourceMac']), emptylist))
    emptylist = [[] for i in range(len(set(window['sourceMac'])))]
    y_errors = dict(zip(set(window['sourceMac']), emptylist))
    
    history = dict(zip(set(window['sourceMac']), np.zeros(len(set(window['sourceMac'])))))
    
    return histos, positions, x_errors, y_errors, history

In the function resetDataStructures all the dictionairies created in createDataStructures are reset: all the MAC addresses get an empty list again.
The dictionairy containing the calculated density estimates is deep copied.

In [7]:
def resetDataStructures(histos, height, width):
    
    histos_old = copy.deepcopy(histos)
    
    grids = np.zeros((len(histos), height,width))
    histos = dict(zip(histos.keys(), grids))
    
    emptylist = [[] for i in range(len(histos))]
    positions = dict(zip(histos.keys(), emptylist))
    emptylist = [[] for i in range(len(histos))]
    x_errors = dict(zip(histos.keys(), emptylist))
    emptylist = [[] for i in range(len(histos))]
    y_errors = dict(zip(histos.keys(), emptylist))
    
    return histos, histos_old, positions, x_errors, y_errors

In the function updateDataStructures all the empty lists in the dictionairies created in createDataStructures are filled with values from the data in the selected time window.
If a MAC address is not yet in the dictionary, it is added.

In [8]:
def updateDataStructures(window, histos, positions, x_errors, y_errors, history, height, width):
    for i in range(len(window)):
        if not window['sourceMac'].values[i] in positions:
            histos[window['sourceMac'].values[i]] = np.zeros((height,width))
            positions[window['sourceMac'].values[i]] = []
            x_errors[window['sourceMac'].values[i]] = []
            y_errors[window['sourceMac'].values[i]] = []
            history[window['sourceMac'].values[i]] = 0
            
        positions[window['sourceMac'].values[i]].append(window['coordinates'].values[i][:2])
        x_errors[window['sourceMac'].values[i]].append(window['x_error'].values[i])
        y_errors[window['sourceMac'].values[i]].append(window['y_error'].values[i])
        
    return histos, positions, x_errors, y_errors, history

## Note:

The function createDensityEstimates below uses the variable_kde class from the dens_estimation module.
We create an instance of the kernel estimator by passing two (2 x N)-arrays to it, containing the N estimated
positions (values), and the N associated uncertainties (errors).
The kernel is then evaluated on a provided set of evaluation grid points (gridpoints). The evaluation grid points 
should be reshaped into a (2 x M)-array (where M is the number of evaluation grid points).
The result (the (1 x M)-vector: estimate) is then reshaped back into the size of the evaluation grid.
So, for each MAC address we pass a list of estimated positions and a list of two-dimensional errors to the kernel. The kernel returns a list of normalized values for the evaluation grid points.
The variable_kde class code is also commented (see dens_estimation.py).

In [9]:
def createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors,  height, width):

    for mac in histos.keys():
        if len(positions[mac]) > 0:
            values = np.transpose(np.array(positions[mac]))
            uncertainties = np.array([x_errors[mac], y_errors[mac]])
            kernel = de.variable_kde(values, uncertainties)
            binvals = kernel(gridpoints)
            # reshape() stacks row-wise, so we use the Fortran-like index ordering
            estimate = np.reshape(binvals, (height,width), order='F')
            histos[mac] += estimate
            
            if histos[mac].sum() > 0:
                histos[mac] /= histos[mac].sum()
    return histos

## Smoothing or memorizing

Please note: below we see two functions, memorizeNonUpdatedEstimates and smoothNonUpdatedEstimates. If the memory parameter > 0,
the smoothNonUpdatedEstimates smoothes non-updated distributions kept in memory. 
If we decide not to smooth non-updated distributions, we should use the function memorizeNonUpdatedEstimates, 
which only repeats distributions as they were last detected.
We can determine what function is used in the function runDataAnalysis.

In [10]:
def memorizeNonUpdatedEstimates(histos, histos_old, positions, history, memory):
    for mac in histos.keys():
        if len(positions[mac]) == 0:
            if history[mac] < memory:
                histos[mac] += histos_old[mac]
                history[mac] += 1
            else:
                history[mac] = 0
    return histos

In the function smoothNonUpdatedEstimates previously calculated density estimates are smoothed.
We first construct a two-dimensional Gaussian bump  of which the width (sigma) is based on by pedestrian walking speed.
The two-dimensional function is created using a scipy.stats library function.
If there were no detections for a MAC address in the time window, its density estimate from the previous time window 
(stored in a deep copy) is convoluted with the Gaussian bivariate bump, using a library function from the scipy signal processing module.
Each time this is done, the history value associated with the MAC address is incremented.
If the history value exceeds the memory parameter value, the density estimate remains zero.
The density estimate remains zero, untill the MAC is detected again.

In [11]:
def get_max_velocity(crowd_density):#by Weidmann's equation
    rho = crowd_density#avoid division by 0
    crowd_density = max(0.1, crowd_density)
    v_0=1.34 #speed at 0 density
    gamma = 1.913 #fit parameter
    rho_max = 5.4 #at this density movement is not possible
    v =v_0*(1-np.exp(-gamma*(1/rho-1/rho_max)))
    v = max(0.01, v)
    #print("velocity:" + str(v))    
    return v

In [12]:
def smoothNonUpdatedEstimates(histos, histos_old, positions, history, height, width, total_dens_histo):
    import math 
    # generate weighting function with dispersion set to 
    # Brownian motion with v = 0.5 m/s and t = interval time
    # diffusion constant D = (v^2)/2
    
    #print("number of people:"+str(np.sum(total_dens_histo)))
    #print("height:" + str(height))
    #print("width:" + str(width))
    crowd_density = np.sum(total_dens_histo)/(stadium_length*stadium_width)
    #print("crowd density:" + str(crowd_density))
    velocity = get_max_velocity(crowd_density)
    #D = velocity*velocity/2
    t = interval / 1000
    #sigma = math.sqrt(2*D*t) / cellsize
    sigma = velocity * math.sqrt(t)/cellsize
    var = multivariate_normal(mean=[width/2 - 1,height/2 - 1], cov=[[sigma**2,0],[0,sigma**2]])
    
    weights = np.zeros((height,width))
    for i in np.arange(width):
        for j in np.arange(height):
            weights[j][i] += var.pdf([i,j])
    
    for mac in histos.keys():
        if len(positions[mac]) == 0:
            if history[mac] < memory:
                # smooth existing pdf from previous time interval
                # apply a convolution
                conv = signal.convolve2d(histos_old[mac], weights, boundary='wrap', mode='same')
                
                histos[mac] += conv
                history[mac] += 1
            else:
                history[mac] = 0
    
    return histos

The sumHistograms function simply sums the histograms in the dictionairy 'histos', 
and returns the total density estimate in the form of a numpy array.

In [13]:
def sumHistograms(histos, height, width):
    # total density histogram per period
    total_dens_histo = np.zeros((height, width))
    
    for mac in histos.keys():
        total_dens_histo += histos[mac]
                
    return total_dens_histo

This is the __main__ function. It runs all the steps, and writes the total density estimate to file.
It differentiates between the first and later iterations, in order to initialize the dictionairies,
and then only to reset them.
Here we also choose whether we smooth non-updated distributions, or just repeat them as they are.

In [14]:
def runDataAnalysis(frame, smooth, outputdir, init_timestamp, timestep, height, width, periods, interval, memory,  gridpoints):
    total_dens_histo = []
    for k in range(periods):
        window = selectWindow(frame, k, init_timestamp, timestep, interval)
        #if len(window) > 0:
        if k < 1:
            histos, positions, x_errors, y_errors, history = createDataStructures(window, height, width)
            histos, positions, x_errors, y_errors, history =\
            updateDataStructures(window, histos, positions, x_errors, y_errors, history, height, width)
            histos = createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors, height, width)
        else:
            histos, histos_old, positions, x_errors, y_errors = resetDataStructures(histos, height, width)
            histos, positions, x_errors, y_errors, history =\
            updateDataStructures(window, histos, positions, x_errors, y_errors, history, height, width)
            histos = createDensityEstimates(window, gridpoints, histos, positions, x_errors, y_errors, height, width)
            if smooth:
                histos = smoothNonUpdatedEstimates(histos, histos_old, positions, history, height, width, total_dens_histo)
            else:    
                histos = memorizeNonUpdatedEstimates(histos, histos_old, positions, history, memory)

        total_dens_histo = sumHistograms(histos, height, width)
        
        np.savetxt(outputdir+'dens_histo_%d.csv' %  k, total_dens_histo, delimiter=',')
        #print('Time window:', k)

In this cell the parameters are set to run runDataAnalysis. The variable 'cellsize' sets the distance between
grid points in the evaluation grid.
The variables height and width follow from dividing the size of the evaluation grid (240x180 meter), 
which is the rectangle containing the Arena stadium, by the cellsize.
The variable 'periods' sets the number of time windows to run the analysis.
Interval is the length of the time window (in milliseconds). Timestep is the amount of time the time window is moved at each iteration.
The memory variable determines the amount of time, measured by the number of time windows,
a density estimate is held in memory.

In [15]:
def run_one():
    reload(de)

# cell size (bin size)
    cellsize = 5;
# size of binned region (number of cells)
    stadium_width = 105
    stadium_length = 70
    width = int(stadium_width/cellsize); height = int(stadium_length/cellsize)

# build the evalation grid, on which to evaluate the kernel estimator
    X, Y = np.mgrid[-int(stadium_width/2):int(stadium_width/2):cellsize,-int(stadium_length/2):int(stadium_length/2):cellsize]
    gridpoints = np.vstack([X.ravel(), Y.ravel()])

# numbers of time intervals
    #periods = 9 #number of time windows
    #timestep = 30000 # stride
    #interval = 60000 # length of time window in ms
    #memory = 9 # in number of time windows
    #init_timestamp = 1436047260000

    runDataAnalysis(True, "F:/Arena_sim_data/output/"  )

## Run for all datasets

In [16]:
filepath = "F:/Arena_sim_data/fake_positions_size_"
crowd_size = 0
step = 5000

# cell size (bin size)
cellsize = 5;
# size of binned region (number of cells)
stadium_width = 105
stadium_length = 70
width = int(stadium_width/cellsize); height = int(stadium_length/cellsize)

# numbers of time intervals
periods = 9 #number of time windows
timestep = 30000 # stride
interval = 60000 # length of time window in ms
memory = 9 # in number of time windows
        

def run_analysis_for_all(filepath, crowd_size, step, init_timestamp = 1436047260000):
    for i in range (0, 16):
        crowd_size += step
        print("crowd_size:" + str(crowd_size))
        data = []
        with open(filepath + str(crowd_size) + ".json") as f:
            data = f.readlines()
        json_lines = []
        for line in data:
            jsline = json.loads(line)
            json_lines.append(jsline)
        global frame    
        frame = pd.DataFrame.from_dict(json_lines)
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
        value = pd.DataFrame(list(frame['value']))
        del frame['value']
# make dataframe of dicts nested in 'trackeeHistory' column
        trackee = pd.DataFrame(list(value['trackeeHistory']))
        del value['trackeeHistory']
        localMac = pd.DataFrame(list(trackee['localMac']))
        localMac.columns = ['localMac']
# make dataframe with a 'coordinates' column
        averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
        coordinates = pd.DataFrame(list(averagecoordinate['avg']))
        averagecoordinate = averagecoordinate.join(coordinates)
        error = pd.DataFrame(list(averagecoordinate['error']))
        errorcoordinates = pd.DataFrame(list(error['coordinates']))
        del errorcoordinates[2]
        errorcoordinates.columns = ['x_error','y_error']
        del averagecoordinate['avg']
        del value['averagecoordinate']
# join dataframes
        frame = frame.join(value.join(averagecoordinate))
        frame = frame.join(errorcoordinates)
        frame = frame.join(localMac)
        del frame['error']
        frame = frame[frame['localMac'] == 0]
        frame = frame.sort_values(by='measurementTimestamp')
        reload(de)        
        # build the evalation grid, on which to evaluate the kernel estimator
        X, Y = np.mgrid[-int(stadium_width/2):int(stadium_width/2):cellsize,-int(stadium_length/2):int(stadium_length/2):cellsize]
        gridpoints = np.vstack([X.ravel(), Y.ravel()])
        runDataAnalysis(frame, False,"F:/Arena_sim_data/output/size_"+ str(crowd_size) + "_", 1436047260000, timestep,height, width, periods, interval, memory,  gridpoints)
        #runDataAnalysis(frame, smooth, outputdir, init_timestamp= 1436047260000, timestep= 30000, height, width, periods, interval, memory

In [17]:
run_analysis_for_all(filepath, crowd_size, step, 1436047260000)

crowd_size:5000
crowd_size:10000
crowd_size:15000
crowd_size:20000
crowd_size:25000
crowd_size:30000
crowd_size:35000
crowd_size:40000
crowd_size:45000
crowd_size:50000
crowd_size:55000
crowd_size:60000
crowd_size:65000
crowd_size:70000
crowd_size:75000
crowd_size:80000


# Code for plotting

The function runDataAnalysis writes results to file. Below we read in those files again in order to plot them. 
Make sure the path and file names are correct.

We first get the maximum occuring density value to set the size of the z-axis in the 3-D plots.

In [ ]:
# check maximum value for z-axis limit

from math import ceil

maxValue = 0

for i in range(periods):
    temp = np.loadtxt('F:/Arena_sim_data/output/dens_histo_%d.csv' % i, delimiter=',').max()
    if temp > maxValue:
        maxValue = temp
        
#maxValue = ceil(maxValue)

In [ ]:
maxValue

In [ ]:
fig = plt.figure(figsize=(16,10))

#col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
col = ['w','r','w','w','w','w'] * height * width
# colors = np.random.choice(col, height*width)

for k in range(periods):
    
    ax = fig.add_subplot(111, projection='3d')

    x_data, y_data = np.meshgrid( np.arange(width),
                                  np.arange(height)*(-1) )

    x_data = x_data.flatten()
    y_data = y_data.flatten()

    z_data = np.loadtxt('F:/Arena_sim_data/output/dens_histo_%s.csv' % k, delimiter=',').flatten()
    #z_data = total_dens_histos[k].flatten()
    ax.set_zlim3d(0, maxValue)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if k < 10:
        number = '000' + str(k)
    elif k > 9:
        number = '00' + str(k)
    elif k > 99:
        number = '0' + str(k)
    plt.savefig('F:/Arena_sim_data/output/dens_histo_%s.png' % number)

#plt.show()

In [ ]:
plt.show()

In [ ]:
X[2]


In [ ]:
z_data[13]

In [ ]:
frame